In [ ]:
!pip install -Uqq fastai

In [ ]:
from fastai.vision.all import *
from fastai import *
from torch import nn, optim
import torch

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
path.ls()

In [ ]:
def label_func(filename):
    return '_'.join(filename.split('_')[:-1])

In [ ]:
dls = ImageDataLoaders.from_name_func(
    path, 
    get_image_files(path/'images'), 
    label_func=label_func, 
    valid_pct=0.2,
    item_tfms=Resize(224),
    batch_tfms=Normalize.from_stats(*imagenet_stats)
)

In [ ]:
pat = r'^(.*)_\d+.jpg'
files = get_image_files(path/'images')
dls = ImageDataLoaders.from_name_re(
    path, 
    files, 
    pat,
    valid_pct=0.2, 
    item_tfms=Resize(224),
    batch_tfms=Normalize.from_stats(*imagenet_stats)
)


In [ ]:
len(dls.train_ds), len(dls.valid_ds)

In [ ]:
dls.show_batch()

In [ ]:
# Get a batch of data
x_b, y_b = dls.one_batch()

In [ ]:
# Instantiate a resnet.
# Switch the pretrained parameter to see 
# what pre-trained activations look like.
resnet = resnet18(pretrained=False).cuda()

In [ ]:
# Get the activations of a conv layer
with hook_output(resnet.conv1) as hook:
    with torch.no_grad():
        _ = resnet(x_b)
        acts = hook.stored

In [ ]:
# Pick an image from the batch and take a look
idx = 0
show_image(x_b[idx], title=y_b[idx])

In [ ]:
# Visualize activations from the first 9 filters
fig = plt.figure(figsize=(10,10))
filter_idxs = range(9)
for i, filter_idx in enumerate(filter_idxs):
    ax = fig.add_subplot(3, 3, 1+i)
    ax.matshow(acts[idx][filter_idx].cpu(), cmap='Greys')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()

In [ ]:
# Look at the dimensionality of the output
print(resnet.fc)
with torch.no_grad():
    print(resnet(x_b).shape)

In [ ]:
del resnet

In [ ]:
cbs = [
    EarlyStoppingCallback(patience=3)
]

learn = cnn_learner( # Instantiate a learner object
    dls, # Pass the dataloaders
    models.resnet34, # Specify the architechture you want to use
    pretrained=False, # Tell it NOT to fetch the ImageNet weights
    cbs=cbs, # Pass the callbacks
    metrics=[accuracy, error_rate] # Pass the metrics you want to see
)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, 1e-3)

In [ ]:
cbs = [
    EarlyStoppingCallback(patience=3)
]

learn = cnn_learner( # Instantiate a learner object
    dls, # Pass the dataloaders
    models.resnet34, # Specify the architechture you want to use
    pretrained=True, # Tell it to fetch the ImageNet weights
    cbs=cbs, # Pass the callbacks
    metrics=[accuracy, error_rate] # Pass the metrics you want to see
)

In [ ]:
learn.model[1]

In [ ]:
print(f"""
There are {learn.dls.c} categories.
If we guessed randomly, we would have aboiut a {100. * 1/learn.dls.c:.02f}% chance of guessing correctly.
""")

In [ ]:
# Find a reasonable learning rate
learn.lr_find()

In [ ]:
LR = 2e-3

In [ ]:
# Train the classifier head
learn.fit_one_cycle(1, LR)

In [ ]:
# Unfreeze the backbone
learn.unfreeze()

In [ ]:
# Trainin everything together
learn.fit_one_cycle(1, lr_max=slice(LR))

In [ ]:
cbs = [
    EarlyStoppingCallback(patience=3)
]

learn = cnn_learner( # Instantiate a learner object
    dls, # Pass the dataloaders
    resnet34, # Specify the architechture you want to use
    pretrained=True, # Tell it to fetch the ImageNet weights
    # loss_func=nn.CrossEntropyLoss(), # Pass the loss function
    cbs=cbs, # Pass the callbacks
    metrics=[accuracy, error_rate] # Pass the metrics you want to see
)

In [ ]:
learn.fine_tune(5, base_lr=LR)

In [ ]:
learn.show_results()

In [ ]:
!pip install -Uqq gradio

In [ ]:
import gradio as gr

In [ ]:
def recognize_pet(img):
    breed, cat_id, logits = learn.predict(img)
    proba = float(logits.softmax(dim=0).max())
    return f"""
    Breed: {breed},
    Probability: {proba:.04f}
    """

In [ ]:
gr.Interface(recognize_pet, inputs=gr.inputs.Image(source='upload'), outputs=gr.outputs.Label()).launch(inline=True)

In [ ]:
def conv_block(in_channels, out_channels, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride),
        nn.PReLU(),
        nn.BatchNorm2d(out_channels),
        nn.Dropout(0.2)
    )

In [ ]:
class ResidualConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        # If the channels or output shape are different, 
        # we need a projection so the channels match
        self.projection = True if in_channels != out_channels or stride > 1 else False
        # The projection is just a 1x1 convolutional layer that changes
        # the number of channels to match out_channels.
        if self.projection:
            self.proj = nn.Conv2d(
                in_channels, 
                out_channels, 
                kernel_size=1, 
                padding=0,
                stride=stride # If the shape changes, this is one way to downsample spatially.
            )

        # Now just add the easy part of the network
        self.conv1 = conv_block(in_channels, out_channels, stride)
        self.conv2 = conv_block(out_channels, out_channels)
        self.final_act = nn.PReLU()

    def forward(self, x):
        # Declare the identity
        identity = x
        # but if we need to do the projection, project x 
        # into a different number of channels.
        if self.projection:
            identity = self.proj(x)
        # Pass the original x through the conv network
        acts = self.conv2(self.conv1(x))
        # Return the identity + the activations of the conv network
        return self.final_act(identity + acts)

In [ ]:
fake_batch = torch.randn(2, 32, 28, 28)

In [ ]:
ResidualConvBlock(32, 32)(fake_batch).shape

In [ ]:
ResidualConvBlock(32, 32, 2)(fake_batch).shape

In [ ]:
ResidualConvBlock(32, 64)(fake_batch).shape